In [1]:
import os
import sys
import re

import numpy as np
import pandas as pd
import scipy as sp
from scipy import signal

import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

%matplotlib inline

In [2]:
sys.path.insert(0, '../src')
from features import *
from utils import *

In [3]:
sns.set(rc={'figure.figsize':(20,5)})

In [4]:
mbit_rate = 1/125000

low_fp = '../data/240p/' 
med_fp = '../data/480p/'
high_fp = '../data/1080p/' 

In [5]:
low_dfs = []
for file in os.listdir(low_fp):
    if file != '.ipynb_checkpoints':
        low_dfs.append(pd.read_csv(low_fp+file))
    
med_dfs = []
for file in os.listdir(med_fp):
    if file != '.ipynb_checkpoints':
        med_dfs.append(pd.read_csv(med_fp+file))
    
high_dfs = []
for file in os.listdir(high_fp):
    if file != '.ipynb_checkpoints':
        high_dfs.append(pd.read_csv(high_fp+file))

In [6]:
# stdoan_low = pd.read_csv(low_fp + 'stdoan-101-action-240p-20201127.csv')
# stdoan_med = pd.read_csv(med_fp + 'stdoan-101-action-480p-20201127.csv')
# stdoan_high = pd.read_csv(high_fp + 'stdoan-101-action-1080p-20201127.csv')

In [7]:
low_ms = []
for df in low_dfs:
    low_ms.append(convert_ms_df(df,True))
    
med_ms = []
for df in med_dfs:
    med_ms.append(convert_ms_df(df,True))
    
high_ms = []
for df in high_dfs:
    high_ms.append(convert_ms_df(df,True))

In [8]:
# low_ms = convert_ms_df(stdoan_low, True)
# med_ms = convert_ms_df(stdoan_med, True)
# high_ms = convert_ms_df(stdoan_high, True)

In [9]:
low_resamples = []
for df in low_ms:
    low_resamples.append(df.resample('500ms', on='Time').sum())
    
med_resamples = []
for df in med_ms:
    med_resamples.append(df.resample('500ms', on='Time').sum())
    
high_resamples = []
for df in high_ms:
    high_resamples.append(df.resample('500ms', on='Time').sum())

In [10]:
# low_resample = low_ms.resample('500ms', on='Time').sum()
# med_resample = med_ms.resample('500ms', on='Time').sum()
# high_resample = high_ms.resample('500ms', on='Time').sum()


## Aggregate Features

In [11]:
## take the aggregate features of the whole chunk; download and upload
def agg_feat(df, col):
    return [np.mean(df[col]), np.std(df[col])]

## take the ratio of upload:download packets
def pkt_ratio(df):
    ms_df = convert_ms_df(df, True)
    local = np.sum(ms_df['pkt_src'] == '1') 
    server = np.sum(ms_df['pkt_src'] == '2') 
    return local / server

## take the ratio of upload:download bytes
def bytes_ratio(df):
    local = df['1->2Bytes'].sum()
    server = df['2->1Bytes'].sum()
    return local / server

## Peak Related Aggregate Features

In [12]:
## finds the peaks with mean + 2(1) std
## run the above aggregate functions on the peaks only??

def get_peak_loc(df, col, invert=False):
  'invert arg allows you to get values not considered peaks'
  df_avg = df[col].mean()
  df_std = df[col].std()
  
  threshold = df_avg + (1 * df_std)
  
  if invert:
    return np.array(df[col] < threshold)
  
  else:
    return np.array(df[col] > threshold)

## np.mean, np.var, np.std - think of more?  
def peak_time_diff(df, col):
  '''
  mess around with the different inputs for function. 
  variance seems to inflate the difference betweent the two the most with litte
  to no data manipulation. however, currently trying things like
  squaring the data before taking the aggregate function to exaggerate
  differences (moderate success??)
  '''
  peaks = df[get_peak_loc(df, col)]
  peaks['Time'] = peaks['Time'] - peaks['Time'].min()
  time_diff = np.diff(peaks['Time'] ** 2)
  return [np.mean(time_diff), np.var(time_diff), np.std(time_diff)]

In [13]:
from scipy.signal import find_peaks

def peak_times(df,col,thresh):
    x = df[col]
    peaks, _ = find_peaks(x, height=thresh)
    if list(peaks) == []:
        return [-1]
    times = df.iloc[peaks]['Time'].values
    time_between_peaks = [times[i]-times[i-1]for i in range(1,len(times))]
    #print(time_between_peaks)
    #time_between_peaks[0]=0
    if time_between_peaks == []:
        return -1
    return time_between_peaks

def num_peaks(df,col,thresh):
    x = df[col]
    peaks, _ = find_peaks(x, height=thresh)
    return len(peaks)

## Spectral Features

In [14]:
def spectral_features(df, col):

    """
    welch implemention of spectral features
    resample the data before inputting (might change prereq depending on
    resource allocation)
    """

    f, Pxx_den = sp.signal.welch(df[col], fs=2)
    Pxx_den = np.sqrt(Pxx_den)

    peaks = sp.signal.find_peaks(Pxx_den)[0]
    prominences = sp.signal.peak_prominences(Pxx_den, peaks)[0]

    idx_max = prominences.argmax()
    loc_max = peaks[idx_max]

    return [f[loc_max], Pxx_den[loc_max], prominences[idx_max]]

## Chunking & Feature creation

In [15]:
## wip; need to decide chunk size eventually
## should we also make this chunking feature be our feature creation?

def chunk_data(df, interval=60):

    """
    takes in a filepath to the data you want to chunk and feature engineer
    chunks our data into a specified time interval
    each chunk is then turned into an observation to be fed into our classifier
    """

    df_list = []
    
    df['Time'] = df['Time'] - df['Time'].min()
    
    total_chunks = np.floor(df['Time'].max() / interval).astype(int)

    for chunk in np.arange(total_chunks):
      
        start = chunk * interval
        end = (chunk+1) * interval

        temp_df = (df[(df['Time'] >= start) & (df['Time'] < end)])
        
        df_list.append(temp_df)
        
    return df_list

In [58]:
def create_features(dfs, interval=60):

  features = [
    'dwl_peak_freq',
    'dwl_peak_prom',
    'dwl_max_psd',
    'dwl_bytes_avg',
    'dwl_bytes_std',
    'dwl_peak_avg',
    'dwl_peak_var',
    'dwl_peak_std',
    'upl_peak_freq',
    'upl_peak_prom',
    'upl_max_psd',
    'upl_bytes_avg',
    'upl_bytes_std',
    'upl_peak_avg',
    'upl_peak_var',
    'upl_peak_std',
      'IMAN_dwn_time_peak',#'IMAN_up_time_peak',
      'IMAN_dwn_num_peak'#,'IMAN_up_num_peak'
  ]  

  vals = []
  for df in dfs:
      df_chunks = chunk_data(df, interval)

      for chunk in df_chunks:

        preproc = convert_ms_df(chunk, True)
        upl_bytes = preproc[preproc['pkt_src'] == '1'].resample('500ms', on='Time').sum()
        dwl_bytes = preproc[preproc['pkt_src'] == '2'].resample('500ms', on='Time').sum()

        ## spectral features
        dwl_spectral = spectral_features(dwl_bytes, 'pkt_size')
        upl_spectral = spectral_features(upl_bytes, 'pkt_size')

        ## aggregate features
        dwl_agg = agg_feat(chunk, '2->1Bytes')
        upl_agg = agg_feat(chunk, '1->2Bytes')

        ## peak features
        dwl_peak = peak_time_diff(chunk, '2->1Bytes')
        upl_peak = peak_time_diff(chunk, '1->2Bytes')
        
        ## iman's time between peak 
        iman_dwn_time_peak = np.mean(peak_times(chunk,'2->1Bytes',1000000))
        #iman_up_time_peak = np.mean(peak_times(chunk,'1->2Bytes',50000))
        
        ## iman's num peak
        iman_dwn_num_peak = num_peaks(chunk,'2->1Bytes',1000000)
        #iman_up_num_peak = num_peaks(chunk,'1->2Bytes',50000)


        
        feat_val = np.hstack((
          dwl_spectral,
          dwl_agg,
          dwl_peak,
          upl_spectral,
          upl_agg,
          upl_peak,
            iman_dwn_time_peak,#iman_up_time_peak,
            iman_dwn_num_peak,#iman_up_num_peak
        ))

        vals.append(feat_val)
    
  return pd.DataFrame(columns=features, data=vals).fillna(0)

## Dev Playground

In [59]:
%%time
low_feat = create_features(low_dfs, 100)
med_feat = create_features(med_dfs, 100)
high_feat = create_features(high_dfs, 100)

CPU times: user 10.8 s, sys: 293 ms, total: 11 s
Wall time: 11.1 s


In [60]:
# %%time
# low_feat = create_features(stdoan_low, 100)
# med_feat = create_features(stdoan_med, 100)
# high_feat = create_features(stdoan_high, 100)

In [61]:
low_feat['resolution'] = np.zeros(len(low_feat))
med_feat['resolution'] = np.zeros(len(med_feat)) + 1
high_feat['resolution'] = np.zeros(len(high_feat)) + 2

In [62]:
training = pd.concat([low_feat, med_feat, high_feat]).reset_index(drop=True)

In [63]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, f1_score,accuracy_score

In [64]:
X, y = training.drop(columns=['resolution']), training['resolution']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 4)

In [65]:
classifier = RandomForestClassifier(n_estimators = 8, criterion = 'entropy', random_state = 42)
classifier.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='entropy',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=8,
                       n_jobs=None, oob_score=False, random_state=42, verbose=0,
                       warm_start=False)

In [66]:
training

,dwl_peak_freq,dwl_peak_prom,dwl_max_psd,dwl_bytes_avg,dwl_bytes_std,dwl_peak_avg,dwl_peak_var,dwl_peak_std,upl_peak_freq,upl_peak_prom,upl_max_psd,upl_bytes_avg,upl_bytes_std,upl_peak_avg,upl_peak_var,upl_peak_std,IMAN_dwn_time_peak,IMAN_dwn_num_peak,resolution
0,0.072165,2.559624e+05,2.283896e+05,114425.905263,305025.605472,820.454545,5.367184e+05,732.610695,0.070707,13224.990587,10583.824498,7366.642105,17675.769254,820.454545,6.460564e+05,803.776356,13.000000,2.0,0.0
1,0.397959,1.608469e+05,1.473483e+05,58461.855072,183186.907760,1479.200000,1.589888e+06,1260.907514,0.397959,10074.840733,9316.483272,3780.942029,9327.334796,1479.200000,1.589888e+06,1260.907514,-1.000000,0.0,0.0
2,0.851064,2.153807e+05,2.039448e+05,40925.519481,156778.675598,900.000000,2.778365e+05,527.101983,0.304569,12063.980132,11423.459269,3293.870130,8072.574910,900.000000,2.778365e+05,527.101983,-1.000000,0.0,0.0
3,0.429319,1.678943e+05,1.543345e+05,51165.656250,212334.491065,1633.333333,1.472686e+06,1213.542839,0.072539,9448.058128,8840.608035,3652.531250,10500.679157,1633.333333,1.472686e+06,1213.542839,-1.000000,1.0,0.0
4,0.666667,1.888234e+05,1.394501e+05,49224.350649,170981.928605,1729.800000,1.760775e+06,1326.941807,0.500000,12246.146287,10209.228504,3438.961039,8511.764942,1729.800000,1.760775e+06,1326.941807,-1.000000,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94,0.100503,1.097268e+06,1.043545e+06,253738.413043,580189.681359,902.500000,3.431702e+05,585.807349,0.100000,53262.247600,52123.889925,13293.500000,27622.507815,902.500000,3.431702e+05,585.807349,10.500000,9.0,2.0
95,0.092308,9.765048e+05,8.906232e+05,217527.819444,516862.726467,882.000000,1.714066e+06,1309.223625,0.092308,46129.569359,41808.893237,11329.333333,24669.122888,882.000000,1.714066e+06,1309.223625,10.400000,6.0,2.0
96,0.183673,5.251847e+05,4.571161e+05,229211.126437,575108.238130,921.600000,8.172820e+05,904.036526,0.183673,25874.543373,24163.202914,11889.528736,27504.462558,921.600000,8.172820e+05,904.036526,13.333333,7.0,2.0
97,0.438776,8.523427e+05,8.051908e+05,221431.108108,578239.976452,435.125000,1.715989e+05,414.244927,0.122449,40051.038123,37570.814535,11485.864865,27438.619912,435.125000,1.715989e+05,414.244927,8.428571,8.0,2.0


In [67]:
y_pred = classifier.predict(X_test)

In [68]:
(pd.crosstab(y_test, y_pred, rownames=['Actual Species'], colnames=['Predicted Species']))

Predicted Species,0.0,1.0,2.0
Actual Species,,,
0.0,9,1,0
1.0,0,8,1
2.0,0,1,5


In [69]:
f1_score(y_test, y_pred, average=None)

array([0.94736842, 0.84210526, 0.83333333])

In [70]:
accuracy_score(y_test, y_pred)

0.88

In [72]:
features = ['dwl_peak_freq','dwl_peak_prom','dwl_max_psd','dwl_bytes_avg','dwl_bytes_std','dwl_peak_avg',
    'dwl_peak_var','dwl_peak_std','upl_peak_freq','upl_peak_prom','upl_max_psd','upl_bytes_avg','upl_bytes_std','upl_peak_avg','upl_peak_var','upl_peak_std',
            'IMAN_dwn_time_peak',#'IMAN_up_time_peak'
            'IMAN_dwn_num_peak']#,'IMAN_up_num_peak']
importances = classifier.feature_importances_
indices = np.argsort(importances)[::-1]
for i in indices:
    print(features[i],': ',importances[i])
    

dwl_bytes_avg :  0.19400570497736236
dwl_peak_prom :  0.11227263932552736
dwl_max_psd :  0.1042636064227472
dwl_bytes_std :  0.08968094200608726
upl_bytes_std :  0.07848357699122407
IMAN_dwn_num_peak :  0.06381409230154074
upl_bytes_avg :  0.06055308573336525
IMAN_dwn_time_peak :  0.05811615118452074
upl_peak_std :  0.0565660596685456
dwl_peak_std :  0.038137768880718986
upl_peak_var :  0.03594913995472983
upl_peak_avg :  0.027938730329112548
upl_max_psd :  0.026433267677344052
dwl_peak_var :  0.022523436077079007
dwl_peak_freq :  0.011267126580791875
upl_peak_freq :  0.010633962739230833
upl_peak_prom :  0.009360709150072312
dwl_peak_avg :  0.0


In [30]:
# ## testing that feature method functions correctly

# l_start = 0 
# l_end = 60

# test_chunk = stdoan_low.copy()
# test_chunk['Time'] = test_chunk['Time'] - test_chunk['Time'].min()
# low_chunk = stdoan_low[(stdoan_low['Time'] >= 0) & (stdoan_low['Time'] < 60)]

# low_chunk_ms = convert_ms_df(low_chunk, True)

# upl_ms = low_chunk_ms[low_chunk_ms['pkt_src'] == '1']
# dwl_ms = low_chunk_ms[low_chunk_ms['pkt_src'] == '2']

# dwl_chunk_rs = dwl_ms.resample('500ms', on='Time').sum()

# f_dwl, Pxx_dwl = sp.signal.welch(dwl_chunk_rs['pkt_size'], fs=2)